In [5]:
import resources as r
import ipywidgets as widgets
from IPython.display import HTML, display
from tqdm.notebook import tqdm_notebook
from copy import deepcopy

layout = widgets.Layout(width='50%')
buttonLayout  = widgets.Layout(min_width='max-content')

def extractIndices(df):
    levels = len(df.index[0])
    indexLists = [[] for i in range(levels)]
    for row in df.index:
        for count, level in enumerate(row):
            if level not in indexLists[count]:
                indexLists[count] += [level] 
    return [df.columns.to_list()] + indexLists

with r.statusLoad(df='progress') as df:
    solvents, fluorophores, states, metajobs = extractIndices(df)

def submit() -> None:
    out.clear_output()
    with out:
        with r.statusLoad(df='progress') as df:
            solvents, fluorophores, states, metajobs = extractIndices(df)
            metajob = jobType_widg.value
            states = deepcopy(list(stateList_widg.value))
            if not metajob.gs:
                try: 
                    states.remove(r.States.s0)
                    print(f'{r.States.s0} not logical with {metajob}')
                except ValueError: pass
            if not metajob.es:
                try: 
                    states.remove(r.States.s1)
                    print(f'{r.States.s1} not logical with {metajob}')
                except ValueError: pass
                try: 
                    states.remove(r.States.s2)
                    print(f'{r.States.s2} not logical with {metajob}')
                except ValueError: pass
            for state in states:
                for fluorophore in fluorophoreList_widg.value:
                    for solvent in solventList_widg.value:
                        if (fluorophore.gas) and (not fluorophore.revised) and (solvent != r.Solvents.gas):
                            # print(f'{fluorophore} is gas-only and cannot be used with {solvent}')
                            pass
                        else:
                            if (solvent != r.Solvents.gas) and (metajob.gasonly == True):
                                print(f'Metajob {metajob} cannot be used with solvent')
                            else:
                                status = df.at[(fluorophore, state, metajob), solvent]
                                refStatus = df.at[(fluorophore, refJobState_widg.value, refJobType_widg.value), solvent]
                                if refStatus == r.Status.finished:
                                    if (status != r.Status.finished) or resubmit_widg.value:
                                        refJob = r.Job.from_MetaJob(refJobType_widg.value, fluorophore, solvent, refJobState_widg.value)
                                        job = r.Job.from_MetaJob(metajob, fluorophore, solvent, state, catxyzpath=refJob.xyzfile, submit=submit_widg.value, 
                                                                    partner=partner_widg.value, procs=cores_widg.value, 
                                                                    mem=memory_widg.value, time=(time_widg.value*24),
                                                                    # ORCA Specific
                                                                    kdiis=orca_kdiis_widg.value, soscf=orca_soscf_widg.value, 
                                                                    notrah=orca_notrah_widg.value, scfstring=orca_scfstring_widg.value)
                                        with r.monarchHandler() as mon:
                                            fileName = mon.buildJob(job)

                                        if submit_widg.value == True:
                                                df.at[(fluorophore, state, metajob), solvent] = r.Status.queued
                                    else:
                                        print(f'{metajob} of state {state}of {fluorophore} in {solvent} already finished')
                                else:
                                    print(f'Reference job: {refJobType_widg.value} of state {state} of {fluorophore} in {solvent} not finished')
        print('Done!')


func = widgets.interactive(submit, {'manual' : True, 'manual_name' : 'Build Job'},)

jobType_widg = widgets.Dropdown(options=metajobs, rows=len(metajobs), value=r.MetaJobs.or_wb_freq_gs, description='Job Type')
refJobType_widg  = widgets.Dropdown(options=metajobs, rows=len(metajobs), value=r.MetaJobs.or_wb_opt_gs, description='Geom Reference Job Type')
refJobState_widg  = widgets.Dropdown(options=states, rows=len(states), value=r.States.s0, description='Geom Reference Job State')
fluorophoreList_widg = widgets.SelectMultiple(options=fluorophores, rows=len(fluorophores), description='Fluorophore')
solventList_widg = widgets.SelectMultiple(options=solvents, rows=len(solvents), description='Solvents')
stateList_widg = widgets.SelectMultiple(options=states, rows=len(states), description='States')

# useBest_widg = widgets.ToggleButton(value=True, icon='check', description='Use S0 if ES not ready', layout=buttonLayout)
# useXD_widg = widgets.ToggleButton(value=True, icon='check', description='Use ωB97X-D opt for other functionals', layout=buttonLayout)
submit_widg = widgets.ToggleButton(value=False, icon='check', description='Submit', layout=buttonLayout)
resubmit_widg = widgets.ToggleButton(value=False, icon='check', description='ReSubmit', layout=buttonLayout)


memory_widg = widgets.IntText(value=64, description='Memory (GB)')
time_widg = widgets.IntText(value=1, description='Job Time (days)')
cores_widg = widgets.IntText(value=16, description='Cores')
partner_widg = widgets.ToggleButton(value=True, icon='check', description='Partner', layout=buttonLayout)

system = widgets.VBox([widgets.HTML('System Settings'),
                       memory_widg, 
                       cores_widg,
                       time_widg
                       ])
submit = widgets.VBox([widgets.HTML('Submission Settings'),
                       submit_widg, 
                       resubmit_widg,
                       partner_widg
                       ])

job = widgets.VBox([widgets.HTML('Job Settings'),
                       refJobType_widg,
                       refJobState_widg,
                       jobType_widg, 
                       fluorophoreList_widg, 
                       solventList_widg, 
                       stateList_widg
                       ])

out = widgets.Output()

orca_kdiis_widg = widgets.ToggleButton(value=False, icon='check', description='KDIIS', layout=buttonLayout)
orca_soscf_widg = widgets.ToggleButton(value=True, icon='check', description='SOSCF', layout=buttonLayout)
orca_notrah_widg = widgets.ToggleButton(value=True, icon='check', description='NoTRAH', layout=buttonLayout)
orca_scfstring_widg = widgets.Text(value='', description='SCF String', layout=widgets.Layout(width='300px'))
orca = widgets.VBox([orca_kdiis_widg ,
                     orca_soscf_widg,
                     orca_notrah_widg,
                     orca_scfstring_widg])

children = [orca]
tab = widgets.Tab()
tab.children = children
tab.set_title(0, 'ORCA')

display(widgets.HBox([system, submit]), widgets.HBox([job, widgets.VBox([widgets.HTML('Program Specific'),tab])]), func, out)
display(HTML('''<style>
    .widget-label { min-width: 25ex; }
</style>'''))


interactive(children=(Button(description='Build Job', style=ButtonStyle()), Output()), _dom_classes=('widget-i…

Output()